# BIN

### Классы
Классы в Python это возможность расширения стандартного набора типов, т.е. возможность написания собственных типов.

**Аналогия с реальным миром:** класс — это техзадание на деталь. В нём написано из каких элементов деталь состоит и для чего может применяться.
Объекты — это детали, произведённые согласно ТЗ. Их может быть очень много, но они все сделаны по одному чертежу и используются в одних целях.

Объектное программирование реализовано во многих языках программирования и в каждом имеет свои особенности. Здесь будем обсуждать только Python.

In [8]:
# Задаём класс, т.е. чертёж объекта, описываем из чего он будет состоять и что уметь делать.
class MyClass:  # MyClass – название нашего нового типа, как int или str
    def __init__(self, v1, v2):  # функция __init__ описывает создание объектов класса. как правило в ней 
        #определяются поля объектов в которых будут храниться данные.
        # в __init__ первым аргументом всегда передаётся self — это конвенциональное название конструироемого объекта
        # когда внутри __init__ мы задаём какие-то поля self через . это означает что потом у созданных объектов мы будем иметь доступ к этим полям
        self.v1 = v1
        self.v2 = v2

    # функции определённые в теле класса называются методами.
    # методы тоже первым аргументом принимают объект от которого вызываются, т.е. self
    # вы с этим уже встречались, например если d = {1:2, 3:4} - словарь, то d.items() это вызов метода items словаря d
    def increase(self, v1a, v2a):
        self.v1 += v1a
        self.v2 += v2a

# теперь, когда класс определён, можно начать создавать его объекты
mc = MyClass(1, 2)   # эта операция создаёт объект mc и затем вызывает его __init__ для заполнения полей. 
# этот вызов расшифровывается как MyClass.__init__(mc, 1, 2)
print(mc, type(mc))  # печатаем объект и его тип
print(mc.v1, mc.v2)  # печатаем поля объекта. мы определили их в __init__ и заполнили значениями в MyClass(1, 2), теперь смотрим их значения

<__main__.MyClass object at 0x111d84350> <class '__main__.MyClass'>
1 2


### Утиная типизация (duck typing)
Утиная типизация заключается в том, что вместо проверки типа чего-либо в Python мы склонны проверять, какое поведение оно поддерживает. Название происходит от т.н. «утиного теста»
```
Если это выглядит как утка, плавает как утка и крякает как утка, то это, вероятно, и есть утка.
```
Несколько утиных типов Python:
- `Sequence` - последовательность. Последовательности состоят из двух основных поведенческих факторов: они имеют длину, и их можно индексировать от 0 до числа, которое меньшей длины последовательности. Среди стандартных типов это список `list`.
- `Iterable` - итерируемый. Обобщение понятия `Sequence`, то есть, все последовательности итерируемы, но не всё что итерируемо является последовательностью. То что можно использовать в циклах; объект, элементы которого можно перебирать. Среди стандартных типов это `list`, `set`, `dict`. В частности, поскольку `set` является неупорядоченным, в нём нет доступа к элементам по индексам, поэтому он не является `Sequence`. Но элементы `set` можно перебирать в цикле `for … in …`, поэтому он `Iterable`.
- `Callable` - вызываемый. Ведёт ли себя объект как функция, т.е. можно ли его вызвать через `()` с какими-нибудь аргументами.
- `Mapping` - отображение. Ведёт ли себя объект как словарь, то есть можно ли ему назначить пары ключ-значение через `[]`.

#### Магические методы (Dunder methods)
Поведение сообразно утиной типизации реализуется в Python c помощью магических методов — название которых начинается и заканчивается двумя подчёркиваниями.
Например, если хочется чтобы объект вёл себя как функция, нужно чтобы он реализовывал метод `__call__()`. Добавление / удаление элементов реализуется с помощью `__getitem__`, `__setitem__`, `__delitem__`.

In [9]:
# Пример: задаём класс, объекты которого можно вызывать как функции

class CustomCallable:
    def __init__(self, what):
        self.what = what

    # сигнатура метода __call__(self, x) означает что можно будет вызывать self(x)
    def __call__(self, x):
        if self.what == 'x**2':
            return x ** 2
        elif self.what == '1/x':
            return 1 / x
        elif self.what == 'x':
            return x
        else:
            return 0
            
my_fn = CustomCallable('x**2')  # создаём объект класса CustomCallable с полем self.what равным 'x**2'
print(2, my_fn(2))  # он реализовывает функцию возведения в квадрат
my_fn.what = '1/x'  # меняем поле what, теперь это будет 1/x
print(2, my_fn(2))
my_fn.what = 'x'  # ещё раз меняем поле what, теперь это будет x
print(2, my_fn(2))
my_fn.what = 'cosine'  # задаём значение не предусмотренное в коде
print(2, my_fn(2))  # теперь на любой аргумент функция вернёт 0

2 4
2 0.5
2 2
2 0


In [10]:
# Пример: задаём класс, объекты которого можно складывать по модулю 9, т.е. a + b это не просто сумма a и b, но остаток от деления этой суммы на 9
class CustomAdditive:
    def __init__(self, value):
        self.modulo = 9
        self.value = value % self.modulo

    # сигнатура __add__(self, other) это тот метод который будет вызываться если к обьекту класса CustomAdditive будет добавляться любой другой обьект
    def __add__(self, other):
        if not isinstance(other, CustomAdditive):
            return self
        return CustomAdditive(self.value + other.value)
    # магический метод __repr__ (сокращение от representation, представление) отвечает за то, что выводится на экран при печати объекта класса
    def __repr__(self):
        return '[%d mod %d]' % (self.value, self.modulo)

a = CustomAdditive(3)
b = CustomAdditive(8)
print(a, b, a + b)


[3 mod 9] [8 mod 9] [2 mod 9]


### Наследование
Традиционно важной частью объектного программирования является наследование. Смысл наследования состоит в следующем:
- Пусть у нас есть класс реализующий какое-то поведение. Например, есть список `list`: в нём можно упорядоченно хранить какие угодно объекты
- Хотим завести класс `ShoppingList`, который вёл бы себя так же как `list`, но при этом умел **что-то дополнительное**: например, хранил отдельным полем название магазина в который надо идти за покупками
- Вместо того чтобы переписывать всю функциональность `list` (что, кстати, просто невозможно) в своём классе, можно написать что `ShoppingList` будет наследником (потомком) класса `list`. То есть? если явно не указано иначе, он будет иметь то же поведение (те же поля и методы) что и `list`. При этом некоторые поля и методы можно добавить, а некоторые переписать (перекрыть).

In [11]:
class ShoppingList(list):  # класс-родитель указывается в скобках после имени определяемого класса
    def __init__(self, name): # если хотим что-то добавить к __init__, т.е. конструктору класса, 
        # мы перекрываем исходный конструктор класса list. Но нам нужно, чтобы функциональность list сохранилась, 
        # поэтому внутри конструктора потомка мы вызываем конструктор предка c ключевым словом super(). 
        # Логика такая: сначала инициируем list, затем придаём ему дополнительные возможности
        super().__init__()
        self.name = name
    #  перекрываем представление объекта так, чтобы перед самим списком возникало имя магазина и двоеточие
    def __repr__(self):
        list_repr = super().__repr__()
        return ': '.join((self.name, list_repr))

shl = ShoppingList('Марийка')
shl.append('Сыр')
shl.append('Каша')
shl.append('Колбаса')
print(shl)

Марийка: ['Сыр', 'Каша', 'Колбаса']
